<a href="https://colab.research.google.com/github/ohilikeit/Dacon_code_similarity/blob/main/bert_base_uncased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/code_similarity

/content/drive/MyDrive/code_similarity


In [ ]:
#!unzip data/open.zip -d data/
!pip install transformers datasets

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('train_data.csv')
data = df.sample(frac=0.1, random_state=42)
sample_train = pd.read_csv('sample_train.csv')
sum_data = pd.concat([data, sample_train], axis = 0)
sum_data.reset_index(drop=True, inplace=True)
sum_data.to_csv('train_2.csv')
del df, data, sample_train, sum_data

In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric

MODEL = "bert-base-uncased"
INPUT = "train_2.csv"
MAX_LEN = 512
dataset = load_dataset("csv", data_files=INPUT)['train']
tokenizer = AutoTokenizer.from_pretrained(MODEL)

def example_fn(examples):
    outputs = tokenizer(examples['code1'], examples['code2'], padding=True, max_length=MAX_LEN,truncation=True)
    if 'similar' in examples:
        outputs["labels"] = examples["similar"]
    return outputs

dataset = dataset.map(example_fn, remove_columns=['code1', 'code2', 'similar'])
dataset = dataset.train_test_split(0.1)

_collator = DataCollatorWithPadding(tokenizer=tokenizer)
_metric = load_metric("glue", "sst2")

def compute_metrics(p):
    preds, labels = p
    output =  _metric.compute(references=labels, predictions=np.argmax(preds, axis=-1),)
    return output
    
model = BertForSequenceClassification.from_pretrained(MODEL) 

args = TrainingArguments(
    'experience',
    evaluation_strategy = 'steps',
    eval_steps = 500,
    load_best_model_at_end = True,
    metric_for_best_model='accuracy', # validation loss
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay= 0.01,
    do_train=True,
    do_eval=True,
    save_strategy='steps',
    logging_strategy="steps"
)

trainer = Trainer(
        model=model,
        args=args,
        data_collator=_collator,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=3)])

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 478212
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train b

Step,Training Loss,Validation Loss,Accuracy
500,0.207800,0.216755,0.924833
1000,0.243400,0.317896,0.879006
1500,0.214900,0.175565,0.937988
2000,0.200500,0.173969,0.943559
2500,0.210100,0.157126,0.943596
3000,0.188400,0.189170,0.931890
3500,0.177000,0.170932,0.949544
4000,0.176900,0.143095,0.948490
4500,0.168800,0.160009,0.953364
5000,0.166300,0.153721,0.948923


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 53135
  Batch size = 16
Saving model checkpoint to experience/checkpoint-500
Configuration saved in experience/checkpoint-500/config.json
Model weights saved in experience/checkpoint-500/pytorch_model.bin
tokenizer config file saved in experience/checkpoint-500/tokenizer_config.json
Special tokens file saved in experience/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5

TrainOutput(global_step=9000, training_loss=0.17455885060628254, metrics={'train_runtime': 23852.6816, 'train_samples_per_second': 60.146, 'train_steps_per_second': 3.759, 'total_flos': 3.737046897038784e+16, 'train_loss': 0.17455885060628254, 'epoch': 0.3})

In [ ]:
import pandas as pd

TEST = "test.csv"
SUB = "sample_submission.csv"

test_dataset = load_dataset("csv", data_files=TEST)['train']
test_dataset = test_dataset.map(example_fn, remove_columns=['code1', 'code2'])

predictions = trainer.predict(test_dataset)

df = pd.read_csv(SUB)
df['similar'] = np.argmax(predictions.predictions, axis=-1)
df.to_csv('experience_submit.csv', index=False)
df

Using custom data configuration default-0873717fbc731c11


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0873717fbc731c11/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/179700 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: pair_id. If pair_id are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 179700
  Batch size = 16


,pair_id,similar
0,1,0
1,2,1
2,3,0
3,4,0
4,5,0
...,...,...
179695,179696,1
179696,179697,1
179697,179698,0
179698,179699,1


In [ ]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

88

In [ ]:
import os
import torch
MODEL_PATH = './best_models'
# 모델의 architecture와 함께 저장
torch.save(trainer, os.path.join('/content/drive/MyDrive/code_similarity', 'model.pt'))

# 모델의 architecture와 함께 load
#model = torch.load(os.path.join(MODEL_PATH, 'model.pt'))